In [58]:
from pathlib import Path
import shutil
import os

location_of_original_data = "../"
from utils import parse_meta_file, parse_text_file
os.system(f"mkdir {location_of_original_data}S_data")
metafiles = Path(location_of_original_data).glob("*_meta.tsv")

def correct_id(s: str) -> str:
    from parse import compile
    
    pattern = "ParlaMint-{lang}_T{term:d}.S{session:d}.{rest}"
    p = compile(pattern)
    
    r = p.parse(s).named
    lang, term, session, rest = r.get("lang"), r.get("term"), r.get("session"), r.get("rest")
    return f"ParlaMint-{lang}_T{term:02}.S{session:02}.{rest}"


for m in metafiles:
    metadf = parse_meta_file(str(m.absolute()))
    textdf = parse_text_file(str(m.absolute()).replace("meta", "text").replace(".tsv", ".txt"))
    term = metadf.Term.unique()
    assert len(term) == 1, "More than one term in file!"
    term = term[0]
    for session in metadf.Session.unique():
        subset = metadf[metadf.Session==session].reset_index(drop=True)
        textsubset = textdf[textdf.ID.isin(subset.ID)].reset_index(drop=True)
        filename = f"meta_T{term:02}_S{session:02}.tsv"
        
        # Skip if we have no texts for session:
        if textsubset.shape[0] == 0:
            continue
        
        # Correcting IDs:
        subset["ID"] = subset.ID.apply(correct_id)
        textsubset["ID"] = textsubset.ID.apply(correct_id)
        # Writing data
        subset.to_csv(f"../S_data/{filename}",sep="\t", index=False)
        with open("../S_data/"+filename.replace("meta", "text").replace("tsv", "txt"), "w") as f:
            f.writelines([
                f"{row['ID']} {row['Text']}" for i, row in textsubset.iterrows()
            ])
